In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/attention_flow

/content/gdrive/MyDrive/attention_flow


In [3]:
!ls

attention_graph_util.py		 graphs			 res_fat_bert.png
bert_example.ipynb		 __pycache__		 res_jat_bert_att_2.png
bert_sv.ipynb			 rat_bert_att_2.png	 res_jat_bert.png
bert_viz.ipynb			 rat_bert_bar_2.png	 res_rat_bert_mrpc.pdf
compute_corel_distilbert_sst.py  rat_bert_mrpc.pdf	 res_rat_bert.png
correlation_logs		 rat_bert.png		 run_bert_sv.py
distilbert_sst.ipynb		 README.md		 save
distill_bert_viz.ipynb		 res_fat_bert_att_2.png  utils


In [24]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import *
import numpy as np
from tqdm import tqdm

from attention_graph_util import *
# from compute_corel_distilbert_sst.py import *
# from notebooks.notebook_utils import *
from utils import inflect

from tqdm import tqdm
import math
import pandas as pd

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [27]:

def spearmanr(x, y):
    """ `x`, `y` --> pd.Series"""
    x = pd.Series(x)
    y = pd.Series(y)
    assert x.shape == y.shape
    rx = x.rank(method='dense')
    ry = y.rank(method='dense')
    d = rx - ry
    dsq = np.sum(np.square(d))
    n = x.shape[0]
    coef = 1. - (6. * dsq) / (n * (n**2 - 1.))
    return [coef]

def get_raw_att_relevance(full_att_mat, input_tokens, layer=-1):
    cls_index = 0
    return full_att_mat[layer].sum(axis=0).sum(axis=0)


def compute_node_flow(G, labels_to_index, input_nodes, output_nodes,length):
    number_of_nodes = len(labels_to_index)
    flow_values=np.zeros((number_of_nodes,number_of_nodes))
    for key in output_nodes:
        if key not in input_nodes:
            current_layer = int(labels_to_index[key] / length)
            pre_layer = current_layer - 1
            u = labels_to_index[key]
            for inp_node_key in input_nodes:
                v = labels_to_index[inp_node_key]
                flow_value = nx.maximum_flow_value(G,u,v)
                flow_values[u][pre_layer*length+v] = flow_value
            #normalize flow values
            flow_values[u] /= flow_values[u].sum()

    return flow_values

def get_flow_relevance(full_att_mat, input_tokens, layer):

    res_att_mat = full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    res_att_mat = res_att_mat + np.eye(res_att_mat.shape[1])[None,...]
    res_att_mat = res_att_mat / res_att_mat.sum(axis=-1)[...,None]

    res_adj_mat, res_labels_to_index = get_adjmat(mat=res_att_mat, input_tokens=input_tokens)

    A = res_adj_mat
    res_G=nx.from_numpy_array(A, create_using=nx.DiGraph())
    for i in np.arange(A.shape[0]):
        for j in np.arange(A.shape[1]):
            nx.set_edge_attributes(res_G, {(i,j): A[i,j]}, 'capacity')

    output_nodes = []
    input_nodes = []
    for key in res_labels_to_index:
        if key.startswith('L'+str(layer+1)+'_'):
            output_nodes.append(key)
        if res_labels_to_index[key] < full_att_mat.shape[-1]:
            input_nodes.append(key)

    flow_values = compute_node_flow(res_G, res_labels_to_index, input_nodes, output_nodes, length=full_att_mat.shape[-1])

    n_layers = full_att_mat.shape[0]
    length = full_att_mat.shape[-1]
    final_layer_attention_raw = flow_values[(layer+1)*length: (layer+2)*length,layer*length: (layer+1)*length]
    relevance_attention_raw = final_layer_attention_raw.sum(axis=0)

    return relevance_attention_raw


def get_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=True)
    relevance_attentions = joint_attentions[layer].sum(axis=0)
    return relevance_attentions


def get_nores_joint_relevance(full_att_mat, input_tokens, layer):
    att_sum_heads =  full_att_mat.sum(axis=1)/full_att_mat.shape[1]
    joint_attentions = compute_joint_attention(att_sum_heads, add_residual=False)
    relevance_attentions = joint_attentions[layer].sum(axis=0)
    return relevance_attentions

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading file chat_template.jinja from cache at None


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.3",
  "vocab_size": 30522
}



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.48.3",
  "vocab_size": 30522
}



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/model.safetensors
Loaded 66,955,010 parameters in the TF 2.0 model.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [8]:
data = tfds.load('glue/sst2')

# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='sst-2')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='sst-2')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)
test_dataset = glue_convert_examples_to_features(data['test'], tokenizer, max_length=128, task='sst-2')
test_dataset = test_dataset.batch(1)

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.evaluate(valid_dataset, steps=100)
# Train and evaluate using tf.keras.Model.fit()
# history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
#                     validation_data=valid_dataset, validation_steps=7)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/sst2/2.0.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.11/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.11/dist-packages/transformers/data/processors/glue.py:330: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
Using label list ['0', '1'] for task sst-2
Using output mode classification for task sst-2
*** Example ***
guid: 16

 14/100 [===>..........................] - ETA: 22s - loss: 0.3902 - accuracy: 0.9106

100/100 [==============================] - 11s 36ms/step - loss: 0.3902 - accuracy: 0.9106


[0.3902028501033783, 0.9105504751205444]

In [14]:
model.config.output_attentions = True
model.config.output_hidden_states = True

model.distilbert.transformer.output_attentions = True
model.distilbert.transformer.output_hidden_states = True

for layer in model.distilbert.transformer.layer:
    layer.output_attentions = True
    layer.output_hidden_states = True
    layer.attention.output_attentions = True
    layer.attention.output_hidden_states = True

all_examples_grads = []
all_examples_attentions = []
all_examples_x = []
n_examples = 10
for x,y in test_dataset:
    with tf.GradientTape() as tape:
        inputs_embeds = model.distilbert.embeddings(x['input_ids'])
        tape.watch(inputs_embeds)
        output = model({'attention_mask':x['attention_mask'],
                                                    'inputs_embeds':inputs_embeds},
                                                    # 'token_type_ids': x['token_type_ids']},
                                                  training=False
                                            )
        logits = output.logits
        hidden_states = output.hidden_states
        attentions = output.attentions
        true_logits = logits[:,y[0]]
        print(true_logits)
    grads = tape.gradient(true_logits, inputs_embeds)[0]

    length = tf.reduce_sum(x['attention_mask'], axis=-1)[0]
    all_examples_grads.append(tf.reduce_sum(grads, -1)[:length])

    _attentions = [att.numpy() for att in attentions]
    attentions_mat = np.asarray(_attentions)[:,0]

    cropped_input = x['input_ids'][0, :length]
    all_examples_x.append(cropped_input)
    cropped_attention_mat = attentions_mat[:,:,:length,:length]
    all_examples_attentions.append(cropped_attention_mat)

    if n_examples == 0:
        break
    n_examples -= 1

tf.Tensor([-3.6605856], shape=(1,), dtype=float32)
tf.Tensor([-3.6501343], shape=(1,), dtype=float32)
tf.Tensor([-3.5526545], shape=(1,), dtype=float32)
tf.Tensor([4.6508036], shape=(1,), dtype=float32)
tf.Tensor([-3.1746943], shape=(1,), dtype=float32)
tf.Tensor([-2.0300615], shape=(1,), dtype=float32)
tf.Tensor([4.5237985], shape=(1,), dtype=float32)
tf.Tensor([-3.2844446], shape=(1,), dtype=float32)
tf.Tensor([4.5983353], shape=(1,), dtype=float32)
tf.Tensor([-3.4095066], shape=(1,), dtype=float32)
tf.Tensor([2.0042334], shape=(1,), dtype=float32)


In [28]:
print("compute raw relevance scores ...")
all_examples_raw_relevance = {}
for l in np.arange(0,6):
    all_examples_raw_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_raw_att_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_raw_relevance[l].append(np.asarray(attention_relevance))

print("compute joint relevance scores ...")
all_examples_joint_relevance = {}
for l in np.arange(0,6):
    all_examples_joint_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_joint_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_joint_relevance[l].append(np.asarray(attention_relevance))

print("compute flow relevance scores ...")
all_examples_flow_relevance = {}
for l in np.arange(5,6):
    all_examples_flow_relevance[l] = []
    for i in tqdm(np.arange(len(all_examples_x))):
        tokens = tokenizer.convert_ids_to_tokens(all_examples_x[i])
        length = len(tokens)
        attention_relevance = get_flow_relevance(all_examples_attentions[i], tokens, layer=l)
        all_examples_flow_relevance[l].append(np.asarray(attention_relevance))

compute raw relevance scores ...


100%|██████████| 11/11 [00:00<00:00, 153.71it/s]


compute joint relevance scores ...


100%|██████████| 11/11 [00:00<00:00, 143.26it/s]


compute flow relevance scores ...


100%|██████████| 11/11 [05:50<00:00, 31.87s/it]


In [29]:
for l in np.arange(0,6):
    print("###############Layer ",l, "#############")

    print('raw grad')
    print(all_examples_raw_relevance[l][0].shape, all_examples_grads[0].shape)
    raw_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_raw_relevance[l][i],all_examples_grads[i])
        raw_sps_grad.append(sp[0])


    print(np.mean(raw_sps_grad), np.std(raw_sps_grad))


    print('joint grad')
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    joint_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_joint_relevance[l][i],all_examples_grads[i])
        joint_sps_grad.append(sp[0])


    print(np.mean(joint_sps_grad), np.std(joint_sps_grad))


    print('flow grad')
for l in np.arange(5,6):
    print(all_examples_joint_relevance[l][0].shape, all_examples_grads[0].shape)
    flow_sps_grad = []
    for i in np.arange(len(all_examples_x)):
        sp = spearmanr(all_examples_flow_relevance[l][i],all_examples_grads[i])
        flow_sps_grad.append(sp[0])

    print(np.mean(flow_sps_grad), np.std(flow_sps_grad))

###############Layer  0 #############
raw grad
(10,) (10,)
0.08659879560137126 0.23706261388605415
joint grad
(10,) (10,)
0.08659879560137126 0.23706261388605415
flow grad
###############Layer  1 #############
raw grad
(10,) (10,)
0.14069492103649578 0.2920799287341343
joint grad
(10,) (10,)
0.10175937039819637 0.258138680184759
flow grad
###############Layer  2 #############
raw grad
(10,) (10,)
0.17256846826587982 0.2762273423253047
joint grad
(10,) (10,)
0.12534749668572878 0.22471524326050285
flow grad
###############Layer  3 #############
raw grad
(10,) (10,)
0.07423142340799543 0.2481214521020669
joint grad
(10,) (10,)
0.13369708433650385 0.19763832197390485
flow grad
###############Layer  4 #############
raw grad
(10,) (10,)
0.060471777739987374 0.33441138647483665
joint grad
(10,) (10,)
0.10856969029379378 0.16829011145858797
flow grad
###############Layer  5 #############
raw grad
(10,) (10,)
0.047778387621607125 0.2948551706526985
joint grad
(10,) (10,)
0.11394132161444998 0.

In [30]:
np.mean([spearmanr(all_examples_flow_relevance[l][i], all_examples_joint_relevance[l][i]) for i in np.arange(len(all_examples_grads))])
np.mean([spearmanr(all_examples_flow_relevance[l][i], all_examples_grads[i]) for i in np.arange(len(all_examples_grads))])

0.07577368100476461

In [31]:
all_examples_joint_relevance

{0: [array([2.31319014, 0.82582609, 0.88103345, 0.83315401, 0.82044793,
         0.86582346, 0.74552972, 0.81991771, 0.78759664, 1.10748085]),
  array([3.77912985, 0.76492161, 0.86228081, 0.86697804, 0.89059019,
         0.87013197, 0.87157969, 0.83322277, 0.92513969, 1.02000379,
         0.98504292, 0.81670259, 0.88045774, 0.8899832 , 0.8671519 ,
         0.88382093, 0.83552041, 0.87371362, 0.93851751, 0.90971339,
         0.84912431, 0.94619006, 0.96086339, 0.92460813, 0.84297364,
         0.77372703, 0.88382689, 1.25408393]),
  array([3.04191166, 0.75548212, 0.81946755, 0.70319781, 0.88094325,
         0.8634736 , 0.79189065, 0.92063738, 0.96064425, 0.92778284,
         0.8938226 , 0.79984255, 0.91901517, 0.87642253, 0.97909606,
         0.87650879, 0.96872409, 0.95475361, 0.95276479, 1.1136187 ]),
  array([2.99583157, 0.80996256, 0.77213441, 0.91689688, 0.86558992,
         0.97700264, 0.88755238, 0.89040515, 0.78697444, 0.81716011,
         0.88815472, 0.88198925, 0.89613482, 0.85